## Get The text from the PDF

In [2]:
%pip install pdfplumber pytesseract pdf2image

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pdfplumber
import pytesseract
from pdf2image import convert_from_path

In [4]:
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        # Try direct text extraction
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text

        if text.strip():
            return text.strip()
    except Exception as e:
        print(f"Direct text extraction failed: {e}")

    # Fallback to OCR for image-based PDFs
    print("Falling back to OCR for image-based PDF.")
    try:
        images = convert_from_path(pdf_path)
        for image in images:
            page_text = pytesseract.image_to_string(image)
            text += page_text + "\n"
    except Exception as e:
        print(f"OCR failed: {e}")

    return text.strip()

In [5]:
pdf_path = "Resume.pdf"
resume_text = extract_text_from_pdf(pdf_path)

print("\nExtracted Text from PDF:")
print(resume_text)

Direct text extraction failed: [Errno 2] No such file or directory: 'Resume.pdf'
Falling back to OCR for image-based PDF.
OCR failed: Unable to get page count. Is poppler installed and in PATH?

Extracted Text from PDF:



## Set Google GenerativeAI Api Key

In [6]:
%pip install google.generativeai python-dotenv

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import google.generativeai as genai
import os
from dotenv import load_dotenv

load_dotenv()
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
model = genai.GenerativeModel("gemini-1.5-flash")

C:\Users\amitn\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
response = model.generate_content("What is the capital of India?")


In [9]:
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "The capital of India is **New Delhi**.\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.013572879135608673
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 7,
        "candidates_token_count": 10,
        "total_token_count": 17
      },
      "model_version": "gemini-1.5-flash"
    }),
)


In [10]:
print(response.text)

The capital of India is **New Delhi**.



## Resume Analysis

In [11]:
def analyze_student_resume(resume_text, job_description=None):
    if not resume_text:
        return {"error": "Resume text is required for analysis."}
    
    model = genai.GenerativeModel("gemini-1.5-flash")
    
    base_prompt = f"""
    You are an experienced HR with Technical Experience in the field of any one job role from Data Science, Data Analyst, DevOPS, Machine Learning Engineer, Prompt Engineer, AI Engineer, Full Stack Web Development, Big Data Engineering, Marketing Analyst, Human Resource Manager, Software Developer your task is to review the provided resume.
    Please share your professional evaluation on whether the candidate's profile aligns with the role. 
    Estimate the ATS score of the resume and provide feedback.
    Concisely list Skills the candidate already has.
    Concisely list the relevant skills the applicant does not possess.
    Concisely Highlight Skills Relevant to the Job Description. Highlight the candidate's strengths and weaknesses. 
    Concisely list a few specific job opportunities the candidate should apply on Jobs Portals, and mention estimated salary per month. 
    Suggest 5 specific projects (and a brief description of technologies) the candidate should make to learn the necessary skills and improve his pre-existing skills.
    Suggest specific courses (in the format Exact Course Name - Course Platform - Skills to Learn) that the candidate should take to improve his skills.
    (Write everything in points and short subpoints, and as many sub-subpoints in the list, as concise as possible.)
    
    Resume:
    {resume_text}
    """

    if job_description:
        base_prompt += f"""
        Additionally, compare this resume to the following job description:
        
        Job Description:
        {job_description}
        
        Highlight the strengths and weaknesses of the applicant in relation to the specified job requirements and give suggestions.
        """

    response = model.generate_content(base_prompt)

    analysis = response.text.strip()
    return analysis

In [12]:
def analyze_candidate_resume(resume_text, job_description=None):
    if not resume_text:
        return {"error": "Resume text is required for analysis."}
    
    model = genai.GenerativeModel("gemini-1.5-flash")
    
    base_prompt = f"""
    You are an experienced HR with Technical Experience in the field of any one job role from Data Science, Data Analyst, DevOPS, Machine Learning Engineer, Prompt Engineer, AI Engineer, Full Stack Web Development, Big Data Engineering, Marketing Analyst, Human Resource Manager, Software Developer your task is to review the provided resume.
    (Write everything the below content in points and short subpoints ,as concise as possible.)
    Concisely mention the applicant's skills, projects, previous experiences, certifications and achievements.
    Concisely mention the skills the applicant does not possess.
    Please share your concise professional evaluation in very short points on whether the candidate's profile aligns with the role. 
    At last give your decision on whether you will accept or reject the candidate.
    
    Resume:
    {resume_text}
    """

    if job_description:
        base_prompt += f"""
        Additionally, compare this resume to the following job description:
        
        Job Description:
        {job_description}
        
        Highlight the strengths and weaknesses of the applicant in relation to the specified job requirements concisely like a HR official.
        """

    response = model.generate_content(base_prompt)

    analysis = response.text.strip()
    return analysis

In [ ]:
def generate_roadmap(resume_text, job_description=None):
    if not resume_text:
        return {"error": "Resume text is required for analysis."}
    
    model = genai.GenerativeModel("gemini-1.5-flash")
    
    base_prompt = f"""
        You are an expert Career Development Coach specializing in {job_description} career paths. Your task is to create a detailed 12-week personalized roadmap for a student based on their resume analysis.

        RESUME ANALYSIS SUMMARY:

        Concisely list the relevant skills the applicant does not possess.
        Concisely Highlight Skills Relevant to the Job Description. 
        Suggest 5 specific projects (and a brief description of technologies) the candidate should make to learn the necessary skills and improve his pre-existing skills.
        Suggest specific courses (in the format Exact Course Name - Course Platform - Skills to Learn) that the candidate should take to improve his skills.

        - Current Skills: 
        - Skills to Develop: 
        - Suggested Projects: 
        - Suggested Courses: 
        - Career Goal: 

        ROADMAP REQUIREMENTS:
        1. Create a structured 12-week learning plan with specific weekly goals
        2. Each week must include:
        - Week number and theme/focus
        - 2-3 specific learning objectives
        - Recommended course modules or lessons (5-10 hours per week)
        - One hands-on mini-project or practical exercise 
        - Required resources and tools
        - Weekend review/assessment task

        3. Progressive difficulty curve:
        - Weeks 1-4: Foundational knowledge and basic skill building
        - Weeks 5-8: Intermediate concepts and applying skills to small projects
        - Weeks 9-12: Advanced topics and portfolio project development

        4. Incorporate the suggested courses and projects from the resume analysis
        5. Include specific milestones to track progress
        6. Estimate time commitment per day (aim for 2-3 hours on weekdays, 3-5 hours on weekends)
        7. Identify 3-5 measurable outcomes to achieve by the end of the roadmap

        FORMAT:
        - Present the roadmap as a structured, easy-to-follow plan
        - Use clear headings for each week
        - Use bullet points for activities and tasks
        - Include motivational elements and progress markers

        The roadmap should be personalized, realistic, and actionable - designed specifically for someone with the candidate's current skill level targeting a {job_description} position.

    Resume:
    {resume_text}
    """

    if job_description:
        base_prompt += f"""
        Additionally, compare this resume to the following job description:
        
        Job Description:
        {job_description}
        
        Highlight the strengths and weaknesses of the applicant in relation to the specified job requirements concisely like a HR official.
        """

    response = model.generate_content(base_prompt)

    analysis = response.text.strip()
    return analysis

In [ ]:
print(generate_roadmap(resume_text))


{'error': 'Resume text is required for analysis.'}
